In [86]:
import os
os.environ["PYSPARK_PYTHON"]="/home/hadoop/anaconda3/bin/python"
from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.charts import Line
from pyecharts.components import Table
from pyecharts.charts import WordCloud
from pyecharts.charts import Pie
from pyecharts.charts import Funnel
from pyecharts.charts import Scatter
from pyecharts.charts import PictorialBar
from pyecharts.options import ComponentTitleOpts
from pyecharts.globals import SymbolType
import json

In [25]:
# 1.每日的累计确诊病例数和死亡数->双柱状图
def drawChart_1(index):
    root = "/home/hadoop/smile/result/result" + str(index) + ".json"
    date = []
    cases = []
    deaths = []
    with open(root, 'r') as f:
        while True:
            line = f.readline()
            if not line:
                break
            js = json.loads(line)
            date.append(str(js['date']))
            cases.append(str(js['cases']))
            deaths.append(str(js['deaths']))
    d = (
    Bar()
        .add_xaxis(date)
        .add_yaxis("累计确诊人数", cases, stack="stack1")
        .add_yaxis("累计死亡人数", deaths, stack="stack1")
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(title_opts=opts.TitleOpts(title="美国每日累计确诊和死亡人数"))
    )
    return d
    
drawChart_1(1).render_notebook()

In [84]:
# 2.每日的新增确诊病例数和死亡数->折线图
def drawChart_2(index):
    root = "/home/hadoop/smile/result/result" + str(index) + ".json"
    date = []
    cases = []
    deaths = []
    with open(root, 'r') as f:
        while True:
            line = f.readline()
            if not line:
                break
            js = json.loads(line)
            date.append(str(js['date']))
            cases.append(int(js['caseIncrease']))
            deaths.append(int(js['deathIncrease']))
    a = (
        Line(init_opts = opts.InitOpts(width = "1600px", height = "800px"))
            .add_xaxis(xaxis_data = date)
            .add_yaxis(series_name = "新增确诊",
                       y_axis = cases,
                       markpoint_opts = opts.MarkPointOpts(
                           data = [opts.MarkPointItem(type_="max", name = "最大值")]
                       ),
                       markline_opts = opts.MarkLineOpts(
                           data = [opts.MarkLineItem(type_="average", name = "平均值")]
                       ),
                      )
            .set_global_opts(
                title_opts = opts.TitleOpts(title = "美国每日新增确诊折线图", subtitle = ""),
                tooltip_opts = opts.TooltipOpts(trigger = "axis"),
                toolbox_opts = opts.ToolboxOpts(is_show = True),
                xaxis_opts = opts.AxisOpts(type_ = "category", boundary_gap = False),
            )
        )
    b = (
        Line(init_opts = opts.InitOpts(width = "1600px", height = "800px"))
            .add_xaxis(xaxis_data = date)
            .add_yaxis(
                series_name = "新增死亡",
                y_axis = deaths,
                markpoint_opts = opts.MarkPointOpts(
                    data = [opts.MarkPointItem(type_ = "max", name = "最大值")]
                ),
                markline_opts = opts.MarkLineOpts(
                    data = [
                                opts.MarkLineItem(type_ = "average", name = "平均值"),
                                opts.MarkLineItem(symbol = "none", x = "90%", y = "max"),
                                opts.MarkLineItem(symbol = "circle", type_ = "max", name = "最高点")
                           ]
                ),
            ).set_global_opts(
                title_opts = opts.TitleOpts(title = "美国每日新增死亡折线图", subtitle = ""),
                tooltip_opts = opts.ToolboxOpts(is_show = True),
                xaxis_opts = opts.AxisOpts(type_ = "category", boundary_gap = False),
            )
        )
    
    
    return a, b


drawChart_2(2)[0].render_notebook()

In [85]:
drawChart_2(2)[1].render_notebook()

In [57]:
# 3.截止5.19，美国各州累计确诊、死亡人数和病死率->表格
def drawChart_3(index):
    root = "/home/hadoop/smile/result/result" + str(index) + ".json"
    allState = []
    with open(root, 'r') as f:
        while True:
            line = f.readline()
            if not line:
                break
            js = json.loads(line)
            row = []
            row.append(str(js['state']))
            row.append(int(js['totalCases']))
            row.append(int(js['totalDeaths']))
            row.append(float(js['deathRate']))
            allState.append(row)
    table = Table()
    
    headers = ["State name", "Total cases", "Total deaths", "Death rate"]
    rows = allState
    table.add(headers, rows)
    table.set_global_opts(title_opts = ComponentTitleOpts(title = "美国各州疫情一览", subtitle=""))
    
    return table

drawChart_3(3).render_notebook()

State name,Total cases,Total deaths,Death rate
New York,357757,28437,0.0795
New Jersey,149037,10586,0.071
Illinois,98298,4399,0.0448
Massachusetts,87925,5938,0.0675
California,83981,3422,0.0407
Pennsylvania,67404,4675,0.0694
Michigan,52337,5017,0.0959
Texas,51080,1405,0.0275
Florida,46936,2051,0.0437
Maryland,41664,2081,0.0499


In [59]:
# 美国确诊最多的10个州->词云图
def drawChart_4(index):
    root = "/home/hadoop/smile/result/result" + str(index) + ".json"
    data = []
    with open(root, 'r') as f:
        while True:
            line = f.readline()
            if not line:
                break
            js = json.loads(line)
            row = (str(js['state']), int(js['totalCases']))
            data.append(row)
            
    a = (
        WordCloud()
            .add("", data, word_size_range = [20, 100], shape = SymbolType.DIAMOND)
            .set_global_opts(title_opts = opts.TitleOpts(title = "美国各州确诊Top10"))
    )
    return a

drawChart_4(4).render_notebook()

In [66]:
# 5.美国死亡最多的10个州->象柱状图
def drawChart_5(index):
    root = "/home/hadoop/smile/result/result" + str(index) + ".json"
    state = []
    totalDeath = []
    with open(root, 'r') as f:
        while True:
            line = f.readline()
            if not line:
                break
            js = json.loads(line)
            state.insert(0, str(js['state']))
            totalDeath.insert(0, int(js['totalDeaths']))
        
    a = (
        PictorialBar()
            .add_xaxis(state)
            .add_yaxis(
                "",
                totalDeath,
                label_opts = opts.LabelOpts(is_show = False),
                symbol_size = 18,
                symbol_repeat = "fixed",
                symbol_offset = [0, 0],
                is_symbol_clip = True,
                symbol = SymbolType.ROUND_RECT,
            )
            .reversal_axis()
            .set_global_opts(
                title_opts = opts.TitleOpts(title = "PictorialBar-美国各州死亡人数Top10"),
                xaxis_opts = opts.AxisOpts(is_show = False),
                yaxis_opts = opts.AxisOpts(
                    axistick_opts = opts.AxisOpts(is_show = False),
                    axisline_opts = opts.AxisLineOpts(
                        linestyle_opts = opts.LineStyleOpts(opacity = 0)    
                    ),
                ),
            )
    )
    
    return a

drawChart_5(5).render_notebook()

In [71]:
# 6.找出美国确诊最少的10个州->词云图
def drawChart_6(index):
    root = "/home/hadoop/smile/result/result" + str(index) + ".json"
    data = []
    with open(root, 'r') as f:
        while True:
            line = f.readline()
            if not line:
                break
            js = json.loads(line)
            row = (str(js['state']), int(js['totalCases']))
            data.append(row)
            
    c = (
        WordCloud()
            .add("", data, word_size_range = [100, 20], shape = SymbolType.DIAMOND)
            .set_global_opts(title_opts = opts.TitleOpts(title = "美国各州确诊最少的10个州"))
    )
    
    return c

drawChart_6(6).render_notebook()

In [72]:
# 7.美国死亡最少的10个州->漏斗图
def drawChart_7(index):
    root = "/home/hadoop/smile/result/result" + str(index) + ".json"
    data = []
    with open (root, 'r') as f:
        while True:
            line = f.readline()
            if not line:
                break
            js = json.loads(line)
            data.insert(0, [str(js['state']), int(js['totalDeaths'])])
            
    a = (
        Funnel()
            .add(
                "State",
                data,
                sort_ = "ascending",
                label_opts = opts.LabelOpts(position = "inside"),
            )
            .set_global_opts(title_opts = opts.TitleOpts(title = ""))
    )
    
    return a

drawChart_7(7).render_notebook()    

In [77]:
# 8.美国的病死率->饼状图
def drawChart_8(index):
    root ="/home/hadoop/smile/result/result" + str(index) + ".json"
    values= []
    with open(root, 'r') as f:
        while True:
            line = f.readline()
            if not line:
                break
            js = json.loads(line)
            if str(js['state']) ==  "USA":
                values.append(["Death(%)", round(float(js['deathRate'])*100, 2)])
                values.append(["No-Death(%)", 100 - round(float(js['deathRate'])*100, 2)])
                
    a = (
        Pie()
            .add("", values)
            .set_colors(["black", "orange"])
            .set_global_opts(title_opts = opts.TitleOpts(title = "全美的病死率"))
            .set_series_opts(label_opts = opts.LabelOpts(formatter = "{b}: {c}"))
    )
    
    return a

drawChart_8(8).render_notebook()